In [12]:
import torch
from torch import nn
import dataset
import evaluate
import models
import torch.utils.data as data
import time
import os
import numpy as np

In [2]:
train_rating = 'Data/ml-1m.train.rating'
test_negative = 'Data/ml-1m.test.negative'
train_data, test_data, user_num ,item_num, train_mat = dataset.load_data(train_rating, test_negative)

In [14]:
num_ng = 4
batch_size = 256
epochs = 20
top_k = 10
out = True
model_path = './models/'
model_type = 'mlp'

In [4]:
train_dataset = dataset.Data(
		train_data, item_num, train_mat, num_ng, True)
test_dataset = dataset.Data(
		test_data, item_num, train_mat, 0, False)

train_loader = data.DataLoader(train_dataset,
		batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = data.DataLoader(test_dataset,
		batch_size=100, shuffle=False, num_workers=0)

In [5]:
def get_layers_config(number_hidden_layers, predictive_factor):
  layers = []
  for i in range(number_hidden_layers):
    layers.append(predictive_factor)
    predictive_factor = predictive_factor * 2
  layers.reverse()
  return layers[0]//2, layers

In [7]:
predictive_factor = 32
number_hidden_layers = 4
embedding_dim, layers = get_layers_config(number_hidden_layers, predictive_factor)
embedding_dim, layers

(128, [256, 128, 64, 32])

In [8]:
model = models.MLP(user_num, item_num, embedding_dim, layers)

In [10]:
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
device = 'cpu'

In [15]:
count, best_hr = 0, 0
for epoch in range(epochs):
	model.train() # Enable dropout (if have).
	start_time = time.time()
	train_loader.dataset.ng_sample()

	for user, item, label in train_loader:
		user = user.to(device)
		item = item.to(device)
		label = label.float().to(device)

		model.zero_grad()
		prediction = model(user, item)
		loss = loss_function(prediction, label)
		loss.backward()
		optimizer.step()
		# writer.add_scalar('data/loss', loss.item(), count)
		count += 1

	model.eval()
	HR, NDCG = evaluate.metrics(model, test_loader, top_k)

	elapsed_time = time.time() - start_time
	print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
			time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
	print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

	if HR > best_hr:
		best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
		if out:
			if not os.path.exists(model_path):
				os.mkdir(model_path)
			torch.save(model, 
				'{}{}.pth'.format(model_path, model_type))

print("End. Best epoch {:03d}: HR = {:.3f}, NDCG = {:.3f}".format(
									best_epoch, best_hr, best_ndcg))
